In [ ]:
# ===============================
# 📦 Montar o Drive
# ===============================
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ===============================
# 📦 Instalar Dependências
# ===============================
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps xformers "trl<0.9.0" peft accelerate bitsandbytes
!pip install transformers datasets trl

In [ ]:
DATA_PATH = "/content/drive/MyDrive/TechChallenge3/"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/TechChallenge3/output/"
MODEL_PATH_DATASET = "/content/drive/MyDrive/TechChallenge3/model/"

In [ ]:
# ===============================
# 🔄 Preparar Dados de Treinamento
# ===============================
import json

SYSTEM_PROMPT = "Responda com a descrição do produto baseado no título fornecido."

def prepare_data(input_path, output_path):
    with open(input_path, "r") as f:
        lines = f.readlines()

    formatted = []
    for line in lines:
        item = json.loads(line)
        title = item.get("title", "").strip()
        content = item.get("content", "").strip()
        if not title or not content:
            continue
        prompt = {
            "instruction": SYSTEM_PROMPT,
            "input": item["title"],
            "output": content
        }
        formatted.append(json.dumps(prompt) + "\n")

    with open(output_path, "w") as f:
        f.writelines(formatted)

prepare_data(f"{DATA_PATH}trn.json", f"{OUTPUT_PATH_DATASET}train_data.jsonl")
prepare_data(f"{DATA_PATH}tst.json", f"{OUTPUT_PATH_DATASET}test_data.jsonl")

In [ ]:
# ===============================
# 🧠 Carregar Modelo com Unsloth
# ===============================
import torch
from unsloth import FastLanguageModel

# Carrega o modelo 4-bit
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/tinyllama-bnb-4bit",
    max_seq_length=1024,  # Reduzido para acelerar
    dtype=None,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 16,
    lora_dropout = 0.1,
    bias = "none",
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],  # padrão do TinyLlama
)

# 🔧 Esse é o ponto onde preparamos o modelo para fine-tuning
model = FastLanguageModel.for_training(model)

In [ ]:
# ===============================
# 🔍 Teste do Modelo ANTES do Fine-Tuning
# ===============================
def gerar_resposta_pretreino(produto):
    prompt = f"### Instrução:\nDescreva o seguinte produto:\n### Produto:\n{produto}\n### Resposta:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 🔬 Exemplo de teste antes do fine-tuning
print("🔍 Resultado ANTES do treino:")
print(gerar_resposta_pretreino("Mog's Family of Cats"))

In [ ]:
# ===============================
# 📚 Carregar Dataset
# ===============================
from datasets import load_dataset

path = f"{OUTPUT_PATH_DATASET}train_data.jsonl"

dataset = load_dataset("json", data_files=path, split="train")
dataset = dataset.shuffle(seed=42).select(range(50000))  # Reduz para 50k exemplos (mais rápido)

In [ ]:
# Define os parâmetros de treinamento
from trl import SFTTrainer
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="model",
    logging_dir="logs",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    learning_rate=2e-5,
    logging_steps=50,
    fp16=True,
    report_to="none",
    save_strategy="epoch",
    save_total_limit=1,
)

# Inicia o treinador com o modelo LoRA
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "output",
    max_seq_length = 1024,
    tokenizer = tokenizer,
    args = training_args,
)

trainer.train()

# ✅ Remove os adapters para uso em inferência
FastLanguageModel.for_inference(model)

In [ ]:
# ===============================
# 💾 Salvar Modelo e Tokenizer para uso posterior
# ===============================
model.save_pretrained(MODEL_PATH_DATASET)
tokenizer.save_pretrained(MODEL_PATH_DATASET)
print("✅ Modelo salvo em:", MODEL_PATH_DATASET)

In [ ]:
# ===============================
# 🧪 Função para Geração de Resposta
# ===============================
def gerar_resposta(produto):
    prompt = f"### Instrução:\nDescreva o seguinte produto:\n### Produto:\n{produto}\n### Resposta:"
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 🔍 Teste de exemplo
print(gerar_resposta("Mog's Family of Cats"))
print(gerar_resposta("Why Don't They Just Quit? DVD Roundtable Discussion"))